In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install datasets

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Specify the path to the saved model
drive_save_path = '/content/drive/MyDrive/llama_model_W1'

# Load the model and tokenizer from Google Drive
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=drive_save_path,  # Path to the saved model on Google Drive
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("Model Loaded")
print(f"Model config: {model.config}")

# Check if CUDA is available
print(f"CUDA Available: {torch.cuda.is_available()}")

Mounted at /content/drive
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Model Loaded
Model config: LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "unsloth/llama-2-7b-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [3]:
import pandas as pd
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["Instruction"]  # This column should contain the instruction
    inputs = examples["Informal Text"]  # The text to be converted to a mind map
    outputs = examples["Academic Text"]  # The actual mind map JSON structure (this should be your target)

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Construct the prompt
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}
pass

import pandas as pd
from datasets import Dataset
# Load your dataset
data = pd.read_excel("NewSet0102.xlsx", engine="openpyxl")
# Convert it into a Hugging Face dataset
dataset = Dataset.from_pandas(data)
# Apply the formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)

# Check the first few entries after the mapping
print("\nTransformed Dataset Preview:")
print(dataset[:3])  # Print first 3 rows after transformation


FileNotFoundError: [Errno 2] No such file or directory: 'NewSet0102.xlsx'

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/118 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 118 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 39,976,960


Step,Training Loss
1,0.502900
2,0.588100
3,0.532200
4,0.593900
5,0.564400
6,0.564000
7,0.511900
8,0.592500
9,0.521400
10,0.625900


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Transform the following informal text into formal academic language, ensuring compliance with IEEE formatting standards. Retain the original meaning and maintain existing citations without introducing new ones :", # instruction
        """so we can be free when AI working[2],i gave this suggestion to oragnizations""", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nTransform the following informal text into formal academic language, ensuring compliance with IEEE formatting standards. Retain the original meaning and maintain existing citations without introducing new ones :\n\n### Input:\nso we can be free when AI working[2],i gave this suggestion to oragnizations\n\n### Response:\nadvances in artificial intelligence (AI) have enabled significant improvements in efficiency and productivity, enabling organizations to optimize operations and reduce costs[2]. This transformation has also introduced new challenges related to ethical considerations and job displacement.</s>']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os

from google.colab import drive
drive.mount('/content/drive')

# Specify the path to save the model in Google Drive
drive_model_path = "/content/drive/MyDrive/llama_model_W3"  # Change "MyDrive" to the desired folder
os.makedirs(drive_model_path, exist_ok=True)  # Create directory if it doesn't exist

# Save the model and tokenizer
model.save_pretrained(drive_model_path)
tokenizer.save_pretrained(drive_model_path)

print(f"Model saved to Google Drive at: {drive_model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to Google Drive at: /content/drive/MyDrive/llama_model_W3


In [31]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained Sentence-BERT model
model2 = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to compute similarity
def compare_semantics_sts(text1, text2):
    embeddings1 = model.encode(text1, convert_to_tensor=True)
    embeddings2 = model.encode(text2, convert_to_tensor=True)

    # Compute cosine similarity
    similarity = util.pytorch_cos_sim(embeddings1, embeddings2)
    return similarity.item()

# Example usage:
output_model1 = "Some scientists say AI might reach human-level intelligence in the next few decades."
output_model2 = " Recent advances in artificial intelligence (AI) suggest that it may achieve human-level intelligence within the next few decades."

similarity_score = compare_semantics_sts(output_model1, output_model2)
print(f"Semantic Similarity: {similarity_score:.4f}")


Semantic Similarity: 0.9195


In [34]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained transformer model and tokenizer
model_name = 'bert-base-uncased'  # Or any other suitable model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model3 = AutoModel.from_pretrained(model_name)

# Function to get embeddings for text
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model3(**inputs).last_hidden_state.mean(dim=1)  # Get the mean of the token embeddings
    return embeddings

# Function to compute cosine similarity between two text outputs
def compare_semantics(text1, text2):
    emb1 = get_embedding(text1)
    emb2 = get_embedding(text2)

    # Compute cosine similarity
    similarity = cosine_similarity(emb1.numpy(), emb2.numpy())
    return similarity[0][0]

# Example usage:
output_model1 = "Some scientists say AI might reach human-level intelligence in the next few decades."
output_model2 = "Recent advances in artificial intelligence (AI) suggest that it may achieve human-level intelligence within the next few decades."

similarity_score = compare_semantics(output_model1, output_model2)
print(f"Cosine Similarity: {similarity_score:.4f}")


Cosine Similarity: 0.8991
